# RE

Инструмент поиска и манипуляций с последовательностями символов в тексте.
Активно используются со времён разработки UNIX.

Брайан Керниган об истории создания утилиты grep, популяризовавшей регулярные выражения (10 минут): https://www.youtube.com/watch?v=NTfOnGZUZDk



## Основы регулярных выражений



<b>Литералы</b>: символы, представляющие самих себя:

    АZ, az, Ая, 09, ! "

__Метасимволы__: символы, заменяющие другие символы или их последовательности:

    \ экранизатор

    ^ начало строки

    $ конец строки

    . любой символ (NB! кроме \n, \r)

    | оператор ИЛИ

    ? ленивый поиск: 1 или 0

    * жадный поиск: 0,...

    + хотя бы один: 1,...

    ( ) группировка

    [ ] символьный класс

    { } квантификатор

__Экранирование__: средство превращения метасимвола в литерал

Экранизатор экранируется экранизатором: \\\

__Символьный класс__ []: любой символ из указанного в квадратных скобках набора: 

*[её]лка → елка, ёлка*

Парсится слева направо.

Эквивалентно : *(е|ё)лка*

__Инвертированный символьный класс__ [^]: любой символ помимо указанного в квадратных скобках набора: 

*[^мп]ир → тир, ыир, 7ир,* но не *мир, пир*

<b>Интервал [x-y]</b>: любой символ из указанного в квадратных скобках интервала:

    [A-z][0-9] → A8, z0, s4

NB! Для русского алфавита:
    
    [А-яЁё]

__Метасимволы-эквиваленты интервалов__:

\\w : любой альфанумерический символ и _

    \w = [А-яЁё0-9_] для текста на кириллице

\\W : любой кроме альфанумерических символов и _

    \W = [^А-яЁё0-9_] для текста на кириллице

\\d : любая цифра

    \d = [0-9]

\\D : любой символ кроме цифр

    \D = [^0-9]
    
\\s : любой символ whitespace (пробел , таб \t, перенос строки \n, вертикальный таб \v, form feed \f, перенос каретки \r)

    \s = [ \f\n\r\t\v]

\\S : любой символ кроме whitespace

    \S = [^ \f\n\r\t\v]

__Символы границ__: указывают на положение помеченных ими символов

    ^ : начало строки
    
*^ком -> '__ком__пьютер'*, но не *'хороший __ком__пьютер', 'теле__ком__'*
    
    $ : конец строки
    
*ком$ -> 'теле__ком__'*, но не *'хороший __ком__пьютер', '__ком__пьютер'*
    
    \b : word boundary
    
*\bком -> '__ком__пьютер', 'хороший __ком__пьютер'*, но не *'теле__ком__'*
    
    \B : не word boundary
    
*\Bком -> 'теле__ком__'*, но не *'__ком__пьютер', 'хороший __ком__пьютер'*

__Квантификаторы__: метасимволы, указывающие на количество допустимых совпадений

    ? : либо 1, либо 0 совпадений
     
*с?ходить -> ходить, сходить*

    + : хотя бы 1 совпадение

*пр+ивет -> привет, пррррррррррррривет*
    
    * : любое количество совпадений, в т.ч. 0



*кр*от -> крот, кот, кррррот*
    
    {х,у} : от х до у совпадений

*\d{1,4} -> 8, 56, 0923, 1084*
    
    {x,} : от х совпадений

*\d{2,} -> 25, 490298238178719713728100219*
    
    {,y} : от 0 до y совпадений

*\d{,5} -> , 98765, 28, 049*

<b>Упражнения для повторения основ регулярных выражений + теория</b>: https://regexone.com/

## Более сложные конструкции с регулярными выражениями

<b>Проверка работы регулярного выражения</b>: https://regex101.com/

<b>Жадная, ленивая, ревнивая(NB! нет в Python re) квантификация</b>

Хотим вытащить тэги из строки
```This is a <tag>quick</tag> test```

    Жадная регулярка: <.+> -> <tag>quick</tag>
    
    Ленивая регулярка: <.+?> -> <tag>, </tag>

<b>Группировка</b>

Объединение последовательностей символов в группы для извлечения с помощью ( ).
Поддерживает квантификаторы.

    Пример: \d{2}\.\d{2}\.\d{4} vs. (\d{2})\.(\d{2})\.(\d{4}) в 14.04.2022


<b>NB! match != group</b>

Если не хотите объединять последовательность в группу (...), возьмите последовательность в (?:...).
    
    Пример: (?:\d{2}[\.]){2}\d{4}
    
Группа хранится в ячейке под своим номером (1-9).

К группе можно обращаться по номеру: \1 = первая группа

    Пример: (\w{2})(\d{6}) для st070508

<b>Модификаторы (флаги)</b>

Меняют поведение интерпретатора.

Основные модификаторы:

    (?m)pattern ИЛИ re.M, re.MULTILINE: строка = line

    (?-m)pattern: строка = string

    (?i)pattern или re.I, re.IGNORECASE: регистр не учитывается при поиске совпадений

    (?s)pattern или re.S, re.DOTALL: метасимвол . включает в себя \n и \r

<b>Комментарии</b>

(?#comment): комментарий к регулярке, который не учитывается интерпретатором.

    Пример: (?#буквенный код)(\w{2})(?#числовой идентификатор)(\d{6}) для st070508

<b>Позитивный/негативный просмотр вперёд/назад
    (Positive/negative lookahead/lookbehind)</b>
    
Последовательность, которая (не) должна предшествовать/следовать за выделяемой, но не входит в match.

Позитивный просмотр вперёд: (?=pattern)

Негативный просмотр вперёд: (?!pattern)

Позитивный просмотр назад: (?<=pattern)

Негативный просмотр назад: (?<!pattern)

    Пример: <tag>tagged, хотим вытащить tag внутри <>

    Позитивный просмотр вперёд: tag(?=>)

    Негативный просмотр вперёд: tag(?!g)

    Позитивный просмотр назад: (?<=<)tag

    Негативный просмотр назад: (?<!>)tag

<b>Условие</b>

(?:(?=если)то|иначе), при этом то и иначе не объединяются в группу.

((?=если)то|иначе), при этом то и иначе объединяются в группу.

    Пример: имеем
        10/28/20
        03/15/2020

    Хотим сделать
        28.10.20
        15.03.2020
        
    (Сложное) решение:
        (\d{2})/(\d{2})/(?:(?=\d{4})(\d{4})|(\d{2})), если отдельные группы 3 и 4
        (\d{2})/(\d{2})/((?=\d{4})\d{4}|\d{2}), если общая группа 3

<b>Задание 1.</b> Изменить формат представления ФИО.

<b>Хорошие упражнения на регулярки</b>: http://play.inginf.units.it/#/

## Python re

<b>Альтернатива</b>: regex

Для задания регулярного выражения: r'pattern' ("raw string")

In [1]:
import re

In [2]:
text = """Я помню чудное мгновенье:
Передо мной явилась ты,
Как мимолетное виденье,
Как гений чистой красоты.
В томленьях грусти безнадежной,
В тревогах шумной суеты,
Звучал мне долго голос нежный
И снились милые черты."""

<b>re.match(шаблон, строка, флаг)</b>: ищет по заданному шаблону только в начале строки.

In [27]:
ya_line = re.match(r'(?im)я(.+)', text)
print(ya_line)

<re.Match object; span=(0, 25), match='Я помню чудное мгновенье:'>


In [34]:
# Возвращает объект класса Match

print(dir(ya_line))

['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'end', 'endpos', 'expand', 'group', 'groupdict', 'groups', 'lastgroup', 'lastindex', 'pos', 're', 'regs', 'span', 'start', 'string']


In [33]:
# Можем посмотреть на пойманные группы.

#group(0) = full match
#group(1-9) = group(1-9) последовательно слева направо

print(ya_line.group(0))
print(ya_line.group(1))

Я помню чудное мгновенье:
 помню чудное мгновенье:


<b>re.search(шаблон, строка, флаг)</b>: ищет по заданному шаблону в любом месте строки (выдаст только первое совпадение с шаблоном).

Возвращает тоже объект класса Match.

In [37]:
like = re.search(r'как.+', text, flags = re.I)
print(like)

<re.Match object; span=(50, 73), match='Как мимолетное виденье,'>


In [38]:
like.group(0)

'Как мимолетное виденье,'

<b>re.findall(шаблон, строка, флаг)</b>: ищет все совпадения с заданным шаблоном в строке.

Возвращает список подстрок, соответствующих шаблону.

In [55]:
m_words = re.findall(r'(?i)\bм.+?\b', text)
#m_words = re.findall(r'(?i)\bм.+\b', text) - жадный поиск
print(m_words)

['мгновенье', 'мной', 'мимолетное', 'мне', 'милые']


<b>re.finditer(шаблон, строка, флаг)</b>: ищет все совпадения с заданным шаблоном в строке.

Возвращает итератор по Match-объектам подстрок, соответствующих шаблону. Полезен, например, если нужно знать, на каких позициях нашлось совпадение.

In [11]:
m_words = re.finditer(r'(?i)\bм.+?\b', text)
for word in m_words:
    print(word.group(0), word.span())

мгновенье (15, 24)
мной (33, 37)
мимолетное (54, 64)
мне (164, 167)
милые (197, 202)


<b>re.sub(шаблон, замена, строка)</b>: заменяет все совпадения с заданным шаблоном в строке на заданное значение. Не inplace.

In [80]:
capitalized_text = re.sub(r'\bм', r'М', text)
print(capitalized_text)

Я помню чудное Мгновенье:
Передо Мной явилась ты,
Как Мимолетное виденье,
Как гений чистой красоты.
В томленьях грусти безнадежной,
В тревогах шумной суеты,
Звучал Мне долго голос нежный
И снились Милые черты.


<b>re.compile(шаблон, флаг)</b>: компилирует шаблон в объект re. Теперь его можно использовать многократно для различных операций, используя на нём вышерассмотренные методы.

In [82]:
m_pattern = re.compile(r'(?i)\bм.+?\b')

In [83]:
m_pattern.search(text)

<re.Match object; span=(15, 24), match='мгновенье'>

In [84]:
m_pattern.findall(text)

['мгновенье', 'мной', 'мимолетное', 'мне', 'милые']

## BeautifulSoup

Библиотека для извлечения данных из файлов HTML и XML.

На простом примере: https://keithgalli.github.io/web-scraping/example.html

(Ссылка на туториал по BS: https://www.youtube.com/watch?v=GjKQ6V_ViQE)

In [86]:
# !pip3 install bs4
# Возможно, потребуется дополнительно установить lxml. Попробуйте импортировать его; если возникли проблемы с импортом, то тогда
# !pip3 install lxml
import requests
from bs4 import BeautifulSoup as bs

In [91]:
# Скачаем содержимое страницы
example = requests.get('https://keithgalli.github.io/web-scraping/example.html')

# Наш суп:
example.content

b'<html>\n<head>\n<title>HTML Example</title>\n</head>\n<body>\n\n<div align="middle">\n<h1>HTML Webpage</h1>\n<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>\n</div>\n\n<h2>A Header</h2>\n<p><i>Some italicized text</i></p>\n\n<h2>Another header</h2>\n<p id="paragraph-id"><b>Some bold text</b></p>\n\n</body>\n</html>\n'

In [93]:
# Передадим полученное содержимое BeautifulSoup
soup = bs(example.content)

# Красивый суп:
print(soup.prettify())

<html>
 <head>
  <title>
   HTML Example
  </title>
 </head>
 <body>
  <div align="middle">
   <h1>
    HTML Webpage
   </h1>
   <p>
    Link to more interesting example:
    <a href="https://keithgalli.github.io/web-scraping/webpage.html">
     keithgalli.github.io/web-scraping/webpage.html
    </a>
   </p>
  </div>
  <h2>
   A Header
  </h2>
  <p>
   <i>
    Some italicized text
   </i>
  </p>
  <h2>
   Another header
  </h2>
  <p id="paragraph-id">
   <b>
    Some bold text
   </b>
  </p>
 </body>
</html>



<b>soup.find()</b>: найдёт первый элемент страницы, соответствующий шаблону (строка или список)

In [95]:
first_header = soup.find('h2')

print(first_header)

<h2>A Header</h2>


<b>soup.find_all()</b>: найдёт все элементы страницы, соответствующие шаблону (строка или список)

In [97]:
headers = soup.find_all(['h1', 'h2'])

print(headers)

[<h1>HTML Webpage</h1>, <h2>A Header</h2>, <h2>Another header</h2>]


<b>Можно задать атрибуты тега:</b>

In [98]:
paragraph = soup.find_all('p', attrs={'id': 'paragraph-id'})

print(paragraph)

[<p id="paragraph-id"><b>Some bold text</b></p>]


<b>Можно вкладывать операции поиска друг в друга:</b>

In [100]:
div = soup.find('div')
header = div.find('h1')

print(header)

<h1>HTML Webpage</h1>


<b>Можно вести поиск по содержимому тега (string):</b>

In [103]:
headers = soup.find_all('h2', string=re.compile('[Hh]eader'))

print(headers)

[<h2>A Header</h2>, <h2>Another header</h2>]


### CSS селекторы в BeautifulSoup: https://facelessuser.github.io/soupsieve/

<b>soup.select_one(): найдёт первый элемент страницы, соответствующий шаблону (CSS selector)</b>

In [106]:
content = soup.select_one('body > h2') # родитель > дочерний элемент

print(content)

<h2>A Header</h2>


<b>soup.select(): вернёт список всех элементов страницы, соответствующих шаблону (CSS selector)</b>

In [109]:
content = soup.select('div ~ p') # все дочерние p после div внутри одного родителя

print(content)

[<p><i>Some italicized text</i></p>, <p id="paragraph-id"><b>Some bold text</b></p>]


In [110]:
bold_text = soup.select('p#paragraph-id b') # все b внутри p с атрибутом paragraph-id

print(bold_text)

[<b>Some bold text</b>]


In [111]:
middle = soup.select('[align=middle]') # все элементы с атрибутом align и его значением middle

print(middle)

[<div align="middle">
<h1>HTML Webpage</h1>
<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>
</div>]


Сложно с select? Работаем с find!

<b>.string: выдаст текст, содержащийся внутри одного элемента</b>

In [112]:
header = soup.find('h2')

print(header.string)

A Header


<b>.get_text(): выдаст текст, содержащийся внутри нескольких элементов</b>

In [115]:
div = soup.find('div')

print(div.prettify()) # сам div
print(div.get_text()) # его содержимое, отображающееся на странице

<div align="middle">
 <h1>
  HTML Webpage
 </h1>
 <p>
  Link to more interesting example:
  <a href="https://keithgalli.github.io/web-scraping/webpage.html">
   keithgalli.github.io/web-scraping/webpage.html
  </a>
 </p>
</div>


HTML Webpage
Link to more interesting example: keithgalli.github.io/web-scraping/webpage.html



<b>['...']: выдаст содержимое элемента по тегу</b>

In [119]:
paragraphs = soup.select('p#paragraph-id')

print(paragraphs[0]['id'])

paragraph-id


In [118]:
link = soup.find('a')

print(link['href'])

https://keithgalli.github.io/web-scraping/webpage.html


<b>Пример<b>: скачать ссылки из Notes и References с Викистатьи "Regular expression" (https://en.wikipedia.org/wiki/Regular_expression) и посмотреть, на что чаще всего ссылались авторы.
    Найдём один очень информативный сайт : )

In [124]:
# Скачаем содержимое страницы с помощью запроса
#NB! Это учебный пример. У Вики есть API, применимый в большинстве аналогичных задач.

re_link = "https://en.wikipedia.org/wiki/Regular_expression"

wiki = requests.get(re_link)

In [125]:
wiki.content # суп!

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Regular expression - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"77ac6a76-686c-4eea-880b-9cf5c19d2c66","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Regular_expression","wgTitle":"Regular expression","wgCurRevisionId":986085857,"wgRevisionId":986085857,"wgArticleId":25717,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Harv and Sfn no-target errors","Wikipedia articles needing page number citations from February 2015","Webarchive template wayback links","Articles wi

In [126]:
# Передадим полученное содержимое BeautifulSoup
soup = bs(wiki.content)

In [127]:
print(soup.prettify()) # красивый суп!

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Regular expression - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"77ac6a76-686c-4eea-880b-9cf5c19d2c66","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Regular_expression","wgTitle":"Regular expression","wgCurRevisionId":986085857,"wgRevisionId":986085857,"wgArticleId":25717,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Harv and Sfn no-target errors","Wikipedia articles needing page number citations from February 2015","Webarchive template wayback links","

In [145]:
# Найдём все элементы со ссылками на цитаты

citation_links = []
cites = soup.find_all('cite')

print(cites)

[<cite class="citation web cs1" id="CITEREFGoyvaerts">Goyvaerts, Jan. <a class="external text" href="https://web.archive.org/web/20161101212501/http://www.regular-expressions.info/tutorial.html" rel="nofollow">"Regular Expression Tutorial - Learn How to Use Regular Expressions"</a>. <i>www.regular-expressions.info</i>. Archived from <a class="external text" href="http://www.regular-expressions.info/tutorial.html" rel="nofollow">the original</a> on 2016-11-01<span class="reference-accessdate">. Retrieved <span class="nowrap">2016-10-31</span></span>.</cite>, <cite class="citation book cs1" id="CITEREFMitkov2003">Mitkov, Ruslan (2003). <a class="external text" href="https://books.google.com/books?id=yl6AnaKtVAkC&amp;pg=PA754" rel="nofollow"><i>The Oxford Handbook of Computational Linguistics</i></a>. Oxford University Press. p. 754. <a class="mw-redirect" href="/wiki/ISBN_(identifier)" title="ISBN (identifier)">ISBN</a> <a href="/wiki/Special:BookSources/978-0-19-927634-9" title="Special

In [159]:
# Извлечём сами ссылки

for match in cites:
    for a in match.find_all('a'):
        citation_links.append(a['href'])

print(citation_links[0], citation_links[-1])

https://web.archive.org/web/20161101212501/http://www.regular-expressions.info/tutorial.html https://web.archive.org/web/20100112232513/http://dev.perl.org/perl6/doc/design/apo/A05.html


In [161]:
# Проблема: wiki-links

for link in citation_links[:10]:
    print(link)

https://web.archive.org/web/20161101212501/http://www.regular-expressions.info/tutorial.html
http://www.regular-expressions.info/tutorial.html
https://books.google.com/books?id=yl6AnaKtVAkC&pg=PA754
/wiki/ISBN_(identifier)
/wiki/Special:BookSources/978-0-19-927634-9
https://web.archive.org/web/20170228030346/https://books.google.com/books?id=yl6AnaKtVAkC&pg=PA754
https://books.google.com/books?id=MDQ_K7-z2AMC&pg=PA98
/wiki/ISBN_(identifier)
/wiki/Special:BookSources/978-1-58488-255-8
https://web.archive.org/web/20170227195128/https://books.google.com/books?id=MDQ_K7-z2AMC&pg=PA98


In [163]:
# Оставим только ссылки на внешние источники

nonwiki_links = [x for x in citation_links if x.startswith('http')]

for link in nonwiki_links[:10]:
    print(link)

https://web.archive.org/web/20161101212501/http://www.regular-expressions.info/tutorial.html
http://www.regular-expressions.info/tutorial.html
https://books.google.com/books?id=yl6AnaKtVAkC&pg=PA754
https://web.archive.org/web/20170228030346/https://books.google.com/books?id=yl6AnaKtVAkC&pg=PA754
https://books.google.com/books?id=MDQ_K7-z2AMC&pg=PA98
https://web.archive.org/web/20170227195128/https://books.google.com/books?id=MDQ_K7-z2AMC&pg=PA98
https://web.archive.org/web/20131205193130/https://www.cs.nmsu.edu/historical-projects/Projects/kleene.9.16.10.pdf
https://www.cs.nmsu.edu/historical-projects/Projects/kleene.9.16.10.pdf
http://www.cs.princeton.edu/courses/archive/spr09/cos333/beautiful.html
https://web.archive.org/web/20201007183137/https://www.cs.princeton.edu/courses/archive/spr09/cos333/beautiful.html


In [196]:
# Извлечём архивированные ссылки и уберём повторы

nonwikiarchive_links = list(set([re.sub(r'https://web\.archive\.org/web/\d{14}/', '', link) for link in nonwiki_links]))

nonwikiarchive_links

['http://infolab.stanford.edu/~ullman/focs.html',
 'https://www.cs.princeton.edu/courses/archive/spr09/cos333/beautiful.html',
 'https://www.postgresql.org/docs/9.3/functions-matching.html',
 'http://perldoc.perl.org/perlre.html',
 'https://books.google.com/books?id=MDQ_K7-z2AMC&pg=PA98',
 'https://perl.plover.com/NPC/',
 'https://books.google.com/books?id=yl6AnaKtVAkC&pg=PA754',
 'https://www.scribd.com/book/15491004/Perl-Best-Practices-Standards-and-Styles-for-Developing-Maintainable-Code',
 'https://doi.org/10.1145%2F364175.364185',
 'https://cs.stackexchange.com/a/40058',
 'https://www.scribd.com/doc/15491004/Perl-Best-Practices',
 'https://github.com/travisdowns/polyregex',
 'http://regex.info/',
 'https://doi.org/10.1109%2FLICS.1991.151646',
 'https://doi.org/10.1002%2Fspe.411',
 'http://pubs.opengroup.org/onlinepubs/009695399/basedefs/xbd_chap09.html',
 'http://vimdoc.sourceforge.net/htmldoc/pattern.html#/%5B%5D',
 'http://dev.perl.org/perl6/doc/design/apo/A05.html',
 'http://ww

In [197]:
# Оставим только адреса самих сайтов

sitename_pattern = re.compile(r'(?<=://).+?(?=/)')
sitenames = []

for l in nonwikiarchive_links:
    sitenames.extend(sitename_pattern.findall(l))

In [198]:
print(sitenames[:5])

['infolab.stanford.edu', 'www.cs.princeton.edu', 'www.postgresql.org', 'perldoc.perl.org', 'books.google.com']


In [199]:
# Уберём www.

source_sites = [re.sub(r"^www\.", "", item) for item in sitenames]

source_sites[:5]

['infolab.stanford.edu',
 'cs.princeton.edu',
 'postgresql.org',
 'perldoc.perl.org',
 'books.google.com']

In [200]:
# Создадим частотный список ссылок

from collections import Counter
site_counter = Counter()

for sn in source_sites:
    site_counter[sn] += 1
    
site_counter.most_common(15)

[('doi.org', 6),
 ('pubs.opengroup.org', 3),
 ('api.semanticscholar.org', 3),
 ('infolab.stanford.edu', 2),
 ('cs.princeton.edu', 2),
 ('postgresql.org', 2),
 ('perldoc.perl.org', 2),
 ('books.google.com', 2),
 ('scribd.com', 2),
 ('cs.stackexchange.com', 2),
 ('catb.org', 2),
 ('laurikari.net', 2),
 ('drops.dagstuhl.de', 2),
 ('regular-expressions.info', 2),
 ('perl.plover.com', 1)]

<b>Отличный ресурс по re</b>: https://www.regular-expressions.info/

<b>Задание 2</b>. Извлечь заголовки новостей из архива, найти все, связанные с коронавирусом.

## Токенизация. Natasha

Бейзлайн по делению на токены: re.findall(\w+|\d+|\p+)

Бейзлайн по делению на предложения: re.split([.?!…])

In [176]:
# !pip3 install natasha

Natasha: продукт Лаборатории анализа данных Александра Кукушкина.
Natasha лучше всего работает на новостных текстах со стандартной пунктуацией (см. документацию: https://nbviewer.jupyter.org/github/natasha/natasha/blob/master/docs.ipynb)

### Razdel

Деление на токены и предложения

In [178]:
from razdel import tokenize, sentenize

In [179]:
article = """МВД инициировало уголовное преследование генерала из Петербурга. Ивана Абакумова подозревают в махинациях со здоровьем

Следственный комитет Петербурга возбудил уголовное дело в отношении генерал-майора внутренней службы Ивана Абакумова. «Фонтанка» узнала, как легко он умудрился придумать себе проблему. И притом в Крыму.

Иван Аббакумов//Андрей Пронин/Интерпресс
По информации «Фонтанки», 13 сентября Следственный комитет Петербурга возбудил уголовное дело по части третьей статьи 327 УК – «Подделка, изготовление или оборот поддельных документов, государственных наград, штампов, печатей или бланков, предоставляющих права или освобождающих от обязанностей». Заметим, что эта норма не предусматривает даже лишение свободы, а лишь ее ограничения до одного года. Однако в данном случае важно имя подозреваемого.  
"""

In [206]:
# Делим текст на токены

tokens = list(tokenize(article))

print(tokens[:5])

tokens[0].text

[Substring(0, 3, 'МВД'), Substring(4, 16, 'инициировало'), Substring(17, 26, 'уголовное'), Substring(27, 40, 'преследование'), Substring(41, 49, 'генерала')]


'МВД'

In [208]:
sentences = list(sentenize(article))
sentences

[Substring(0,
           64,
           'МВД инициировало уголовное преследование генерала из Петербурга.'),
 Substring(65,
           237,
           'Ивана Абакумова подозревают в махинациях со здоровьем\n\nСледственный комитет Петербурга возбудил уголовное дело в отношении генерал-майора внутренней службы Ивана Абакумова.'),
 Substring(238,
           304,
           '«Фонтанка» узнала, как легко он умудрился придумать себе проблему.'),
 Substring(305, 322, 'И притом в Крыму.'),
 Substring(324,
           661,
           'Иван Аббакумов//Андрей Пронин/Интерпресс\nПо информации «Фонтанки», 13 сентября Следственный комитет Петербурга возбудил уголовное дело по части третьей статьи 327 УК – «Подделка, изготовление или оборот поддельных документов, государственных наград, штампов, печатей или бланков, предоставляющих права или освобождающих от обязанностей».'),
 Substring(662,
           763,
           'Заметим, что эта норма не предусматривает даже лишение свободы, а лишь ее ограничен

<b>Посмотрим на часть пайплайна Natasha</b>

In [212]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

In [214]:
emb = NewsEmbedding()
segmenter = Segmenter() #обёртка Razdel

morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [215]:
# Делаем из текста объект класса Doc и парсим его

doc_article = Doc(article)

doc_article.segment(segmenter)
doc_article.tag_morph(morph_tagger)
doc_article.parse_syntax(syntax_parser)

In [ ]:
# Смотрим, что получилось

In [220]:
dir(doc_article)

['__annotations__',
 '__attributes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_pretty_',
 'as_json',
 'clear_envelopes',
 'envelop_sent_spans',
 'envelop_sent_tokens',
 'envelop_span_tokens',
 'from_json',
 'morph',
 'ner',
 'parse_syntax',
 'segment',
 'sents',
 'spans',
 'syntax',
 'tag_morph',
 'tag_ner',
 'text',
 'tokens']

In [219]:
vars(doc_article.sents[0])

{'start': 0,
 'stop': 64,
 'text': 'МВД инициировало уголовное преследование генерала из Петербурга.',
 'tokens': [DocToken(stop=3, text='МВД', id='1_1', head_id='1_2', rel='nsubj', pos='PROPN', feats=<Inan,Nom,Neut,Sing>),
  DocToken(start=4, stop=16, text='инициировало', id='1_2', head_id='1_0', rel='root', pos='VERB', feats=<Perf,Neut,Ind,Sing,Past,Fin,Act>),
  DocToken(start=17, stop=26, text='уголовное', id='1_3', head_id='1_4', rel='amod', pos='ADJ', feats=<Inan,Acc,Pos,Neut,Sing>),
  DocToken(start=27, stop=40, text='преследование', id='1_4', head_id='1_2', rel='obj', pos='NOUN', feats=<Inan,Acc,Neut,Sing>),
  DocToken(start=41, stop=49, text='генерала', id='1_5', head_id='1_4', rel='nmod', pos='NOUN', feats=<Anim,Gen,Masc,Sing>),
  DocToken(start=50, stop=52, text='из', id='1_6', head_id='1_7', rel='case', pos='ADP'),
  DocToken(start=53, stop=63, text='Петербурга', id='1_7', head_id='1_4', rel='nmod', pos='PROPN', feats=<Inan,Gen,Masc,Sing>),
  DocToken(start=63, stop=64, text

In [221]:
# Можно красиво распечатать морфоразметку

doc_article.sents[0].morph.print()

                 МВД PROPN|Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
        инициировало VERB|Aspect=Perf|Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
           уголовное ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Neut|Number=Sing
       преследование NOUN|Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing
            генерала NOUN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                  из ADP
          Петербурга PROPN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   . PUNCT


In [222]:
# И синтаксическую разметку тоже

doc_article.sents[0].syntax.print()

      ┌► МВД           nsubj
┌───┌─└─ инициировало  
│   │ ┌► уголовное     amod
│ ┌─└►└─ преследование obj
│ │ └──► генерала      nmod
│ │   ┌► из            case
│ └──►└─ Петербурга    nmod
└──────► .             punct


<b>NER</b>

In [225]:
from natasha import NewsNERTagger

In [226]:
#для простого текста нужен только emb, Doc должен быть сегментирован 

ner_tagger = NewsNERTagger(emb)

In [227]:
# Делаем разметку именованных сущностей

doc_article.tag_ner(ner_tagger)
doc_article.ner.print()

#хранятся в .spans

МВД инициировало уголовное преследование генерала из Петербурга. Ивана
ORG                                                  LOC───────  PER──
 Абакумова подозревают в махинациях со здоровьем
──────────                                      
Следственный комитет Петербурга возбудил уголовное дело в отношении 
ORG───────────────── LOC───────                                     
генерал-майора внутренней службы Ивана Абакумова. «Фонтанка» узнала, 
                                 PER────────────   ORG─────          
как легко он умудрился придумать себе проблему. И притом в Крыму.
                                                           LOC── 
Иван Аббакумов//Андрей Пронин/Интерпресс
PER───────────  PER────────── PER───────
По информации «Фонтанки», 13 сентября Следственный комитет Петербурга 
               ORG─────               ORG───────────────── LOC─────── 
возбудил уголовное дело по части третьей статьи 327 УК – «Подделка, 
изготовление или оборот поддельных документов, государств

<b>Нормализация именованных сущностей</b>

In [234]:
morph_vocab = MorphVocab() #обёртка для pymorphy2

In [235]:
#Доступно для Doc после ner и объявления морфословаря

for span in doc_article.spans:
    span.normalize(morph_vocab)
    
{span.text: span.normal for span in doc_article.spans}

{'МВД': 'МВД',
 'Петербурга': 'Петербург',
 'Ивана Абакумова': 'Иван Абакумов',
 'Следственный комитет': 'Следственный комитет',
 'Фонтанка': 'Фонтанка',
 'Крыму': 'Крым',
 'Иван Аббакумов': 'Иван Аббакумов',
 'Андрей Пронин': 'Андрей Пронин',
 'Интерпресс': 'Интерпресс',
 'Фонтанки': 'Фонтанка'}

<b>Извлечение фактов</b>

In [236]:
# Работает с голым текстом или с Doc, у которого есть spans (с или без нормализации)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
#money_extractor = MoneyExtractor(morph_vocab)
#addr_extractor = AddrExtractor(morph_vocab)

In [242]:
# Извлечение имён

for span in doc_article.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)
    
{_.normal: _.fact.as_dict for _ in doc_article.spans if _.fact}

{'Иван Абакумов': {'first': 'Иван', 'last': 'Абакумов'},
 'Иван Аббакумов': {'first': 'Иван', 'last': 'Аббакумов'},
 'Андрей Пронин': {'first': 'Андрей', 'last': 'Пронин'}}

In [244]:
# Извлечение дат

list(dates_extractor(article))

[Match(
     start=391,
     stop=402,
     fact=Date(
         year=None,
         month=9,
         day=13
     )
 )]

Для MoneyExtractor и AddrExtractor смотрите документацию: https://nbviewer.jupyter.org/github/natasha/natasha/blob/master/docs.ipynb